# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './model/iML1515.xml' 
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)

# Reaction-kcat_mw file.
# eg. AADDGT,49389.2889,40.6396,1215.299582180927
reaction_kcat_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw=pd.read_csv(reaction_kcat_file, index_col=0)

#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5 
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

#Construct an enzyme-constrained model
model = set_enzyme_constraint(model, reaction_kcat_mw, lowerbound, upperbound)

# Simulating overflow metabolism of E. coli

In [3]:
#growth_rate from 0.1 to 0.7 (1/h)
overflow_result = pd.DataFrame()
growth_rate = 0.1
while growth_rate <= 0.7:
    with model as overflow_model:
        overflow_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds = (0.0, 0.0)
        overflow_model.reactions.get_by_id('EX_glc__D_e').bounds = (-1000.0, 0.0)
        overflow_model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').bounds = (growth_rate, growth_rate)
        
        overflow_model.objective = 'EX_glc__D_e'
        pfba_solution = cobra.flux_analysis.pfba(overflow_model)
        overflow_result.loc[growth_rate, 'glucose'] = -pfba_solution.fluxes['EX_glc__D_e']
        overflow_result.loc[growth_rate, 'ac'] = pfba_solution.fluxes['EX_ac_e']
        overflow_result.loc[growth_rate, 'o2_reverse'] = pfba_solution.fluxes['EX_o2_e_reverse']
        growth_rate = growth_rate + 0.1
overflow_result.to_csv("./analysis/pfba_overflow_result.csv")

# Predicting E. coli’s maximum growth rate using different carbon sources

In [4]:
substrates = ['EX_acgam_e', 'EX_ac_e', 'EX_akg_e', 'EX_ala__L_e', 'EX_fru_e', 
        'EX_fum_e', 'EX_g6p_e', 'EX_gal_e', 'EX_gam_e', 'EX_glcn_e', 
        'EX_glc__D_e', 'EX_glyc_e', 'EX_gsn_e', 'EX_lac__L_e', 'EX_malt_e', 
        'EX_mal__L_e', 'EX_man_e', 'EX_mnl_e', 'EX_pyr_e', 'EX_rib__D_e', 
        'EX_sbt__D_e', 'EX_succ_e', 'EX_tre_e', 'EX_xyl__D_e']
growth = pd.DataFrame()
for substrate in substrates:
    with model as growth_model: 
        growth_model.reactions.get_by_id('EX_dha_e').bounds = (0.0, 0.0) 
        growth_model.reactions.get_by_id('EX_pyr_e').bounds = (0.0, 0.0) 
        growth_model.reactions.get_by_id('EX_5dglcn_e').bounds = (0.0, 0.0) 
        growth_model.reactions.get_by_id('EX_xan_e').bounds = (0.0, 0.0) 
        growth_model.reactions.get_by_id('EX_fum_e').bounds = (0.0, 0.0) 
        growth_model.reactions.get_by_id('EX_succ_e').bounds = (0.0, 0.0)
        growth_model.reactions.get_by_id('EX_for_e').bounds = (0.0, 0.0)
        growth_model.reactions.get_by_id('EX_glcn_e').bounds = (0.0, 0.0)
        growth_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds =(0.0, 0.0) 
        growth_model.reactions.get_by_id(substrate).bounds = (-1000.0, 0.0)
        pfba_solution = cobra.flux_analysis.pfba(growth_model)
        growth.loc[substrate, 'pfba_flux'] = pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
        growth.loc[substrate, 'sub_flux'] = pfba_solution.fluxes[substrate]
growth.to_csv("./analysis/growth_pfba.csv")